## Dati Satellite Landsat7 (multispectral bands)

In [1]:
from path import Path
import arrow
import json
import pytz
from pprint import pprint
from tqdm.notebook import tqdm
import re, os, collections, itertools, uuid, logging
import tempfile

import zipfile
import urllib

import ee
import pyproj
import numpy as np
import scipy as sp
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (15, 5) # bigger plots
plt.style.use('fivethirtyeight')
%matplotlib inline
%precision 4

'%.4f'

In [2]:
helper_dir = str(Path('..').abspath())
if helper_dir not in os.sys.path:
    os.sys.path.append(helper_dir)
    
from leak_helpers.earth_engine import display_ee, get_boundary, tifs2np, bands_s2, download_image, bands_s1, bands_l7, bands_l8

In [3]:
# # Non voglio stampare i Warning
# import warnings
# warnings.filterwarnings("ignore")

# Load leaks

In [4]:
# load 
root = "C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite"
leaks = gpd.read_file(root+"/data/leak_dataset/leaks.geojson")

leaks_datas = [leaks]

leaks_datas

[          id  anno civico       comune  \
 0          0  2015    137        AULLA   
 1          1  2015      8        AULLA   
 2          2  2015    110        AULLA   
 3          3  2015    snc        aulla   
 4          4  2015    snc        aulla   
 ...      ...   ...    ...          ...   
 33179  33179  2021   None  GIUNCUGNANO   
 33180  33180  2021   None    MOLAZZANA   
 33181  33181  2021   None    MASSAROSA   
 33182  33182  2021   None    MASSAROSA   
 33183  33183  2021   None    MASSAROSA   
 
                                              description  diametro  \
 0                               Perdita acqua dalla rete      63.0   
 1                               Perdita acqua dalla rete      40.0   
 2                               Perdita acqua dalla rete      90.0   
 3      perdita rete idrica su strada provinciale in l...      90.0   
 4       perdita rete idrica in loc albiano via molinetti      63.0   
 ...                                                  ..

In [5]:
# join them all, with primary columns and random metadata
primary_cols = ['workorderid','reportdate','geometry']
leaks = gpd.GeoDataFrame(pd.concat([leaks_data[primary_cols] for leaks_data in leaks_datas]), crs='epsg:4326')
leaks['metadata'] = np.concatenate([leaks_data.drop(primary_cols,1).to_dict('records') for leaks_data in leaks_datas])
leaks.index = leaks.workorderid
leaks

C:\Users\TRANFA~1.CAR\AppData\Local\Temp/ipykernel_1056/3025104378.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  leaks['metadata'] = np.concatenate([leaks_data.drop(primary_cols,1).to_dict('records') for leaks_data in leaks_datas])


,workorderid,reportdate,geometry,metadata
workorderid,,,,
430398,430398,2015-05-08T08:16:00+00:00,POINT (9.90141 44.17144),"{'id': '0', 'anno': 2015, 'civico': '137', 'co..."
454859,454859,2015-06-10T08:18:00+00:00,POINT (9.91343 44.17462),"{'id': '1', 'anno': 2015, 'civico': '8', 'comu..."
455210,455210,2015-06-11T09:09:00+00:00,POINT (9.90498 44.17221),"{'id': '2', 'anno': 2015, 'civico': '110', 'co..."
459545,459545,2015-06-18T06:00:56.737000+00:00,POINT (9.90778 44.17321),"{'id': '3', 'anno': 2015, 'civico': 'snc', 'co..."
467973,467973,2015-06-30T06:10:52.959999+00:00,POINT (9.91327 44.17462),"{'id': '4', 'anno': 2015, 'civico': 'snc', 'co..."
...,...,...,...,...
2725108,2725108,2021-10-21T14:50:35.837002+00:00,POINT (10.23105 44.20313),"{'id': '33179', 'anno': 2021, 'civico': None, ..."
2730580,2730580,2021-10-26T11:23:17.657000+00:00,POINT (10.41085 44.08091),"{'id': '33180', 'anno': 2021, 'civico': None, ..."
2746652,2746652,2021-11-08T15:39:00+00:00,POINT (10.36244 43.84858),"{'id': '33181', 'anno': 2021, 'civico': None, ..."


## Params

Customise the values in the cell below

In [6]:
# params
bands = bands_l7
satellite = 'LANDSAT/LE7_L1T'
resolution_min = 15.0 # m
cloudy_percentage = 30

# since the lowest res band is 60m and I want to capture neighbours I should get 6+ pixels
pixel_length = 25.0

# you need to tweak this until you pass the "Test the distance need to get your rectangle" cell
fudge_distance_factor = -0.5

## Init

In [9]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "notebook_name = " + "'"+thename+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [10]:
#notebook_name='scraping_earth_engine_landsat7'
notebook_name

'scraping_earth_engine_landsat7'

In [11]:
# constant params, probobly don't change
#time_bin_delta = 60*60*6 # how long before a leak to look (in seconds)
time_before = 6*60*60
time_after = 21*24*60*60
crs_grid = 3857 # keep this as auxilary sphere, this is the CRS the downloaded images will be in

# init
## init directories
ts=arrow.utcnow().format('YYYYMMDD-HH-mm-ss')
temp_dir = Path(root+'/data/scraped_satellite_images/'+satellite.replace("/","_")+'/tmp/')
output_dir = Path(root+'/data/scraped_satellite_images/'+satellite.replace("/","_"))
cache_dir = Path(output_dir+'/cache')
output_dir.makedirs_p()
temp_dir.makedirs_p()
cache_dir.makedirs_p()

## init logger
logger = logging.getLogger(notebook_name)
# logger.setLevel(logging.WARN)

temp_dir, output_dir, cache_dir

(Path('C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/tmp/'),
 Path('C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T'),
 Path('C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache'))

In [12]:
metadata_file = output_dir.joinpath('script_metadata.json')

# write metadata to json
# metadata = dict(
#     pixel_length=pixel_length,
#     resolution_min=resolution_min,
#     bands=bands,
#     ts=ts,
#     notebook_name=notebook_name,
#     crs_grid=crs_grid,
#     cache_dir=str(cache_dir),
#     temp_dir=str(temp_dir),
#     output_dir=str(output_dir),
# )
# json.dump(metadata, open(metadata_file,'w'))

metadata = dict(
    notebook_name=notebook_name,
    satellite=satellite,
    #time_bin_delta=time_bin_delta,
    time_before=time_before,
    time_after=time_after,
    cloudy_percentage=cloudy_percentage,
    pixel_length=pixel_length,
    resolution_min=resolution_min,
    bands=bands,
    ts=ts,
    crs_grid=crs_grid,
    cache_dir=str(cache_dir),
    temp_dir=str(temp_dir),
    output_dir=str(output_dir),
)
metadata_file = output_dir.joinpath('script_metadata.json')
json.dump(metadata, open(metadata_file,'w'))

# Earth Engine

Setup instructions here
- first need to apply for an account and wait ~ 1day
- https://developers.google.com/earth-engine/python_install#setting-up-authentication-credentials

Refs:
- api https://developers.google.com/earth-engine/
- code examples https://code.earthengine.google.com/
- sentinel1 https://developers.google.com/earth-engine/sentinel1
    - `ee.ImageCollection('COPERNICUS/S2_GRD');`
    - `ee.ImageCollection('COPERNICUS/S1_GRD');`
- keras and google earth https://github.com/patrick-dd/landsat-landstats

In [13]:
# test earth-engine setup
from oauth2client import crypt # should have not error
import ee
ee.Initialize() # should give no errors, if so follow instructions


# test
image = ee.Image(ee.ImageCollection(satellite).first())
info = image.getInfo()
info

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [9091, 9011],
   'crs': 'EPSG:32628',
   'crs_transform': [30, 0, 338385, 0, -30, 8810115]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [9091, 9011],
   'crs': 'EPSG:32628',
   'crs_transform': [30, 0, 338385, 0, -30, 8810115]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [9091, 9011],
   'crs': 'EPSG:32628',
   'crs_transform': [30, 0, 338385, 0, -30, 8810115]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [9091, 9011],
   'crs': 'EPSG:32628',
   'crs_transform': [30, 0, 338385, 0, -30, 8810115]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max':

# Fetching images

For a leak repair, grab the image before and after it

Note roughly 10% have results for a 1 day temporal bin
For each point
- find the nearest image before the repair
- and the soonest image after repair
- save a part of each with metadata

Later we can filter, interpolate, and read into numpy arrays

In [14]:
import dataset
cache_file = 'sqlite:///{}'.format(cache_dir.dirname().joinpath('cache.db'))
db = dataset.connect(cache_file)
cache_table = db.get_table('cached_ids', primary_id='workorderid')

def get_cached_ids():
    return set(row['workorderid'] for row in cache_table.distinct('workorderid'))

def init_cache(workorderid):
    """We will cache downloads in folders like 'id_after'"""
    if workorderid:
        try:
            cache_table.insert(dict(workorderid=workorderid))
        except:
            db.rollback()
        else:
            db.commit()
    return

#Aggiunge il workorderid per il quale ho già l'immagine
img_path = Path('../../data/scraped_satellite_images/'+satellite.replace("/","_")+'/cache/')
for i in os.listdir(img_path):
    init_cache(i.split('_')[0])

# Conta il set di workorderid già scaricati
len(get_cached_ids())

562

In [15]:
# # # Cancella dati dalla tabella
#cache_table.delete()

# Conta il set di workorderid che mancano da provare a scaricare
leak_to_scrape = set(leaks.workorderid).difference(set(get_cached_ids()))

len(leak_to_scrape)
#leak_to_scrape

32622

### Test the distance need to get your rectangle

Here we need to tweak `fudge_distance_factor` so that we get the image size of our choice. Start with zero and try -1, -0.5, -.25,0,0.25,0.5,0.75. This is to deal with rounding, projecting between CRS's etc. Don't worry the asserts below will yet you know when it's right.

Occasionaly the problem might be that the leak is at the edge of the image, giving a cropped image. Ignore these rare cases.

In [16]:
distance = resolution_min*(pixel_length/2.00+fudge_distance_factor)

In [19]:
import time
import traceback
cached_ids = get_cached_ids()


def get_image_for_leak(i, cached_ids=cached_ids):
    leak = leaks.loc[[i]]
    repo_date_ts = arrow.get(leak.reportdate.values[0]).timestamp()
    
    # crappy way or recording that we tried this one
    workorderid = leak.workorderid.values[0]
    if workorderid in cached_ids:
        logger.info('Skipping cached download for leak id %s ',workorderid)
        return
    
    boundary = get_boundary(leak, distance=distance)
    
    # get image day before    
    sentinel2_before = (ee.ImageCollection(satellite)
                        .filterBounds(boundary)
                        .filterDate((repo_date_ts-time_before)*1000,(repo_date_ts+time_before)*1000)
                        #.filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',cloudy_percentage)
                        .sort('system:time_start', opt_ascending=False) # first will be latest
                       )
    
    results = sentinel2_before.size().getInfo()
    if results<1:
        logger.info('Error no results for day before %s',workorderid)
        cached_ids = init_cache(str(workorderid)) # so we know there where no results
        return
        
    # get image day after
    sentinel2_after = (ee.ImageCollection(satellite)
                       .filterBounds(boundary)
                       .filterDate((repo_date_ts+time_after)*1000,(repo_date_ts+time_after*2)*1000)
                       #.filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',cloudy_percentage)
                       .sort('system:time_start', opt_ascending=True) # first will be earliest
                      )
        
    results = sentinel2_after.size().getInfo()
    if results<1:
        logger.info('Error no results for day after, id %s',workorderid)
        cached_ids = init_cache(str(workorderid)) # so we know there where no results
        return
        
    # download as save images    
    logger.info('results for %s', workorderid)
    image = ee.Image(sentinel2_before.first()).clip(boundary)
    name=str(workorderid)+'_before'
    path,files=download_image(
        image, 
        scale=resolution_min, 
        crs=crs_grid, 
        name=name,
        cache_dir=cache_dir
    )
    # also save metadata so we can filter by date
    with open(path.joinpath('metadata.json'), 'w') as fo:
        metadata = dict(
            image=image.getInfo(),
            scale=resolution_min,
            crs=crs_grid,
            name=name,
            distance=distance,
            leak=json.loads(leak.to_json())
        )
        json.dump(metadata, fo)

    image = ee.Image(sentinel2_after.first()).clip(boundary)
    name=str(workorderid)+'_after'
    path,files=download_image(
        image, 
        scale=resolution_min, 
        crs=crs_grid, 
        name=name,
        cache_dir=cache_dir
    )
    with open(path.joinpath('metadata.json'), 'w') as fo:
        metadata = dict(
            image=image.getInfo(),
            scale=resolution_min,
            crs=crs_grid,
            name=name,
            distance=distance,
            leak=json.loads(leak.to_json())
        )
        json.dump(metadata, fo)
    cached_ids = init_cache(str(workorderid)) # so we know there where results
    return

leak_to_scrape = set(leaks.workorderid).difference(set(cached_ids))
for i in tqdm(leak_to_scrape):
    try:
        get_image_for_leak(i)
    except urllib.error.HTTPError as e:
        print(i,e) # "HTTP Error 429: unknown"
        traceback.print_stack()
        if e.code == 429:
            print('sleep for 13s')
            time.sleep(13);
    except ee.ee_exception.EEException as e:
        print(i,e) # "Earth Engine memory capacity exceeded."
        traceback.print_stack()
        ee.Initialize()
    except zipfile.BadZipFile as e:
        print(i,e) # "File is not a zip file"
        traceback.print_stack()
    except Exception as e:
        print(i,e)
        traceback.print_stack()

  0%|          | 0/32412 [00:00<?, ?it/s]

# Load Tiffs to Arrays

In [25]:
# This loads it as X and y for machine learning, and also time and metadata so we can filter
import shapely
X = []
y = []
t = []
m = []
discarded=[]
for path in tqdm(cache_dir.listdir()):
    files = [file.relpath(path) for file in path.listdir() if file.endswith('.tif')]
    if files:
        # check metadata
        try:
            metadata = json.load(open(path.joinpath('metadata.json')))
        except (FileNotFoundError, ValueError) as e:
            path.move(path.replace(path.basename(),'.deleteme-'+str(uuid.uuid4())))
            if '_after_' in path: # also delete the before path
                path_after = Path(path.replace('_after_','_before_'))
                if path_after.isdir():
                    path_after.move(path.replace(path.basename(),'.deleteme-'+str(uuid.uuid4())))
            logger.error('Invalid metadata.json, deleted folder %s, please rerun scraping cell to rescrape this image', path)
            continue
        
        # e.g. lets filter it so "before" image are only 1 day before
        if '_before_' in path.basename():
            yy = True
        else:
            yy = False
        
        # work out time gap too
        t1 = arrow.get(metadata['image']['properties']['system:time_end']/1000)
        t0 = arrow.get(metadata['leak']['features'][0]['properties']['reportdate'])
        td=t1-t0
        tt = td.total_seconds()
        
        # load data
        data = tifs2np(path,files,bands=bands)
             
        # check we don't have empty bands 1-13
        empty_bands = np.array([d.sum() for d in data])==0
        
        # lets check we didn't get the edge of an image
        bbox = np.array(metadata['image']['properties']['system:footprint']['coordinates'][0])
        loc = metadata['leak']['features'][0]['geometry']['coordinates']
        minx=bbox[:,0].min()
        maxx=bbox[:,0].max()
        miny=bbox[:,1].min()
        maxy=bbox[:,1].max()
        bbox_shp = shapely.geometry.box(
            minx=minx,
            maxx=maxx,
            miny=miny,
            maxy=maxy
        )
        loc_shp = shapely.geometry.Point(loc[0],loc[1])
        shapely.geometry.GeometryCollection([bbox_shp, loc_shp])
        try:
            print(data.shape)
            assert loc_shp.intersects(bbox_shp), 'leak location should be inside image'
            assert bbox_shp.centroid.almost_equals(loc_shp, decimal=5), 'leak should be near center of image'
            assert (np.array([d.shape for d in data])==pixel_length).all(), 'image area should be the right amount of pixels'
            #assert (maxx-minx)/(maxy-miny)<1.3, 'should be roughly square'
            #assert (maxx-minx)/(maxy-miny)>0.7, 'should be roughly square'
            assert not empty_bands.all(), 'should not have all bands empty'
        except Exception as exc:
            print(path, exc)
#             raise(exc)
            discarded.append(path)
        else:
            X.append(data)
            y.append(yy)
            t.append(tt)
            m.append(metadata)
        

len(X), len(discarded)

  0%|          | 0/1630 [00:00<?, ?it/s]

C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1000273_after_3857_15.0\LE71920292017090NSG00.B1.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1000273_after_3857_15.0\LE71920292017090NSG00.B2.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1000273_after_3857_15.0\LE71920292017090NSG00.B3.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1000273_after_3857_15.0\LE71920292017090NSG00.B4.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1000273_after_3857_15.0\LE71920292017090NSG00.B5.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1000273_after_3857_15.0\LE71920292017090NSG00.B6_VCID_1.tif
C:/Users/tranfa

C:\Users\TRANFA~1.CAR\AppData\Local\Temp/ipykernel_1056/591201699.py:59: ShapelyDeprecationWarning: The 'almost_equals()' method is deprecated and will be removed in Shapely 2.0
  assert bbox_shp.centroid.almost_equals(loc_shp, decimal=5), 'leak should be near center of image'


C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1000431_after_3857_15.0\LE71920302017090NSG00.B8.tif
(9, 25, 25)
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1000431_before_3857_15.0\LE71920302017058NSG00.B1.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1000431_before_3857_15.0\LE71920302017058NSG00.B2.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1000431_before_3857_15.0\LE71920302017058NSG00.B3.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1000431_before_3857_15.0\LE71920302017058NSG00.B4.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1000431_before_3857_15.0\LE71920302017058NSG00.B5.tif
C:/Us

C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1017552_before_3857_15.0\LE71920302017074NSG00.B5.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1017552_before_3857_15.0\LE71920302017074NSG00.B6_VCID_1.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1017552_before_3857_15.0\LE71920302017074NSG00.B6_VCID_2.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1017552_before_3857_15.0\LE71920302017074NSG00.B7.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1017552_before_3857_15.0\LE71920302017074NSG00.B8.tif
(9, 25, 25)
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\1017559_after_3857_15.0\LE71930292017097NSG0

...

C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\902286_after_3857_15.0\LE71920292016360NSG00.B7.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\902286_after_3857_15.0\LE71920292016360NSG00.B8.tif
(9, 25, 25)
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\902286_before_3857_15.0\LE71930292016335NSG01.B1.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\902286_before_3857_15.0\LE71930292016335NSG01.B2.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\902286_before_3857_15.0\LE71930292016335NSG01.B3.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\902286_before_3857_15.0\LE71930292016335NSG01.B4.tif
C:/Users/tra

C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\918162_before_3857_15.0\LE71920302016344NSG00.B2.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\918162_before_3857_15.0\LE71920302016344NSG00.B3.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\918162_before_3857_15.0\LE71920302016344NSG00.B4.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\918162_before_3857_15.0\LE71920302016344NSG00.B5.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\918162_before_3857_15.0\LE71920302016344NSG00.B6_VCID_1.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\918162_before_3857_15.0\LE71920302016344NSG00.B6_VCID_2.tif
C:/Users

C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\948047_after_3857_15.0\LE71930292017049NSG00.B6_VCID_2.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\948047_after_3857_15.0\LE71930292017049NSG00.B7.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\948047_after_3857_15.0\LE71930292017049NSG00.B8.tif
(9, 25, 25)
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\948047_before_3857_15.0\LE71920302017010NSG00.B1.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\948047_before_3857_15.0\LE71920302017010NSG00.B2.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\948047_before_3857_15.0\LE71920302017010NSG00.B3.tif
C:/Use

C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\955311_before_3857_15.0\LE71930292017017NSG00.B4.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\955311_before_3857_15.0\LE71930292017017NSG00.B5.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\955311_before_3857_15.0\LE71930292017017NSG00.B6_VCID_1.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\955311_before_3857_15.0\LE71930292017017NSG00.B6_VCID_2.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\955311_before_3857_15.0\LE71930292017017NSG00.B7.tif
C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T/cache\955311_before_3857_15.0\LE71930292017017NSG00.B8.tif
(9, 25, 

(1572, 58)

In [26]:
# shuffle
from sklearn.utils import shuffle
X,y,m,t = shuffle(X,y,m,t,random_state=1337)

In [27]:
# save using hdf5 (so keras can easily load it) and json 
import h5py
h5file = output_dir.joinpath('data.h5')
with h5py.File(h5file, 'w') as h5f:
    h5f.create_dataset('X', data=X)
    h5f.create_dataset('y', data=y)
    h5f.create_dataset('t', data=t)

json.dump(m,open(output_dir.joinpath('data_metadata.json'),'w'))

with open(output_dir.joinpath('readme.md'),'w') as fo:
    fo. write("""
Files:
- ee_ee_scraping_earth_engine_sentinel_2-austin_leaks- cached tiff files
- script_metadata.json - information on scraping script
- data.h5 contains X, y, and t.
    - X: tiff files for each band loaded into an array of shape (Leak, Bands, width, length)
    - y: True for before the leak, False for after
    - t: time before leak (can be negative) in seconds
- data_metadata: array of metadata for each leak in X. Each contain info on leak, image, and image search
    
Loading: 
```py
# load
metadatas = json.load(open('data_metadata.json'))
with h5py.File('data.h5','r') as h5f:
    X2 = h5f['X'][:]
    y2 = h5f['y'][:]
    t2 = h5f['t'][:]
y
```
    """)

In [28]:
# test load
metadatas = json.load(open(output_dir.joinpath('data_metadata.json')))
with h5py.File(output_dir.joinpath('data.h5'),'r') as h5f:
    X2 = h5f['X'][:]
    y2 = h5f['y'][:]
    t2 = h5f['t'][:]
X2.shape, y2, t2, metadatas[0].keys()

((1572, 9, 25, 25),
 array([ True,  True, False, ..., False,  True,  True]),
 array([  10664.171,    6170.387, 1991302.907, ..., 1997600.08 ,
          -3809.414,    5104.558]),
 dict_keys(['image', 'scale', 'crs', 'name', 'distance', 'leak']))

In [29]:
output_dir

Path('C:/Users/tranfa.carmine/Desktop/Satellite/ricerca_perdite/data/scraped_satellite_images/LANDSAT_LE7_L1T')